# Analysis of contingency tables

## Libraries and settings

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2
from scipy.stats import chi2_contingency

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Get current working directory
print(os.getcwd())

## Import the apartment data

In [ ]:
# Define columns
columns = ['web-scraper-order',
            'address_raw',
            'lat',
            'lon',
            'bfs_number',
            'bfs_name',
            'rooms', 
            'area', 
            'luxurious', 
            'price', 
            'price_per_m2',
            'pop_dens',
            'frg_pct',
            'mean_taxable_income',
            'dist_supermarket']

# Read and select variables
df_orig = pd.read_csv("apartments_data_enriched_cleaned.csv", sep=";")[columns]

# Rename variable 'web-scraper-order' to 'apmt_id'
df_orig = df_orig.rename(columns={'web-scraper-order': 'apartment_id'})

# Remove missing values
df = df_orig.dropna()
df.head(5)

# Remove duplicates
df = df.drop_duplicates()
df.head(5)

## Create a new categorical variable based on the price_per_m2

### Plot histogram of price_per_m2

In [ ]:
# Plot Histogram
fig = plt.figure( figsize=(7,4))
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
n, bins, patches = plt.hist(x=df['price_per_m2'], 
                            bins=20, 
                            color='#42AD12',
                            alpha=0.5, 
                            rwidth=0.95
                   )
plt.grid(True)
plt.ticklabel_format(style='plain')
plt.grid(axis='y', alpha=0.75)

# Set labels
plt.xlabel('price_per_m2', fontsize=10, labelpad=10)
plt.ylabel('Frequency', fontsize=10, labelpad=10)
plt.title('Histogram of price_per_m2', fontsize=12, pad=10)

# Set fontsize of tick labels
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.show()

### Show quantiles of price_per_m2

In [ ]:
qt_price = df['price_per_m2'].quantile(q=[0.10, 0.25, 0.50, 0.75, 0.90]).round(1)
qt_price

### Create new categorical variable 'price_cat'

In [ ]:
# Create labels
labels = ['0 - 19', '20 - 24', '25 - 29', '30 - 34', '>= 35']

# Create new categorical variable
df["price_cat"] = pd.cut(df.price_per_m2, bins=[0, 20, 25, 30, 35, 150], labels=labels)

# Check values
df[['price_per_m2', 'price_cat']].head(10)

## Create a new categorical variable based on the living area

### Plot histogram of area

In [ ]:
# Plot Histogram
fig = plt.figure( figsize=(7,4))
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
n, bins, patches = plt.hist(x=df['area'], 
                            bins=20, 
                            color='blue',
                            alpha=0.5, 
                            rwidth=0.95
                   )
plt.grid(True)
plt.ticklabel_format(style='plain')
plt.grid(axis='y', alpha=0.75)

# Set labels
plt.xlabel('area (m2)', fontsize=10, labelpad=10)
plt.ylabel('Frequency', fontsize=10, labelpad=10)
plt.title('Histogram of area', fontsize=12, pad=10)

# Set fontsize of tick labels
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.show()

### Show quantiles of area

In [ ]:
qt_area = df['area'].quantile(q=[0.10, 0.25, 0.50, 0.75, 0.90]).round(1)
qt_area

### Create new categorical variable 'area_cat'

In [ ]:
# Create labels
labels = ['0 - 49', '50 - 74', '75 - 99', '100 - 149', '>= 150']

# Create new categorical variable
df["area_cat"] = pd.cut(df['area'], bins=[0, 50, 75, 100, 150, 1000], labels=labels)

# Check values
df[['area', 'area_cat']].head(10)

## Create a contingency table with the number of apartments per 'area_cat' versus 'price_cat'

In [ ]:
# Contingency table (numbers in the table are apartments)
tab = pd.pivot_table(df[['apartment_id', 'area_cat', 'price_cat']],
                     index=['area_cat'],
                     columns=['price_cat'], 
                     aggfunc='count')
tab

## Perform a Chi-square test on the contingency table
For details see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html

In [ ]:
# 1-alpha
prob = 0.95

# Chi-squared test
stat, p, dof, expected = chi2_contingency(tab)

# Critical value
critical = chi2.ppf(prob, dof)

# Print results of the Chi-square test
print('Expected frequencies:', '\n', expected.round(4), '\n')
print('Critical value:', critical.round(4), '\n')
print('Test statistic:', stat.round(4), '\n')
print('Degrees of freedom:', dof, '\n')
print(f'p-value: {p:.4f}')

## Interpretation of the Chi-squared test result

<p>Because the test statistic is larger than the critical value and the p-value is smaller than our significance level apha=0.05, we can reject the null hypothesis, i.e. there is a statistically significant (at the 0.05 significance-level) association between the price-categories and the living-area categories in the contingence table.</p>

## Applying the Chi-squared test to animal behavior

For full study see: https://davida-rosenstrauch.medium.com/applying-the-chi-squared-test-to-animal-behavior-b6cca3f930b8

<img src="https://miro.medium.com/max/640/1*h5JgsNbUy2tuiLgSrkIlyA.png" width="500" align="left"/>

### Contingency table, spotted hyena example

In [ ]:
# The numpy array shows how many times each hyena responded (see study above)
table = np.array([[0,4],
                  [7,6],
                  [9,0],
                  [15,10]])

# Create a data frame from 'table'
df_hy = pd.DataFrame(table, columns=[['Female', 'Male']],
             index=['Fear', 'Greeting', 'Aggression', 'No Response'])
df_hy

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')